In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# **Cleaning Data Conclusions**

## Day 2
- `final_demo_df`
- `final_experiment_clients_df`

# Final Demo DF

In [4]:
final_demo_df = pd.read_csv('../../data/clean/final_demo_df_clean.csv')
final_demo_df.head()

,client_id,client_tenure_years,client_tenure_months,client_age,gender,num_accounts,balance,calls_last_6_months,logons_last_6_months
0,836976,6,73,60,Unspecified,2,45105.30,6,9
1,2304905,7,94,58,Unspecified,2,110860.30,6,9
2,1439522,5,64,32,Unspecified,2,52467.79,6,9
3,1562045,16,198,49,Male,2,67454.65,3,6
4,5126305,12,145,33,Female,2,103671.75,0,3


### Table Overview

**Columns:**

- `client_id (int)`: A unique identifier for each client, used to distinguish one client from another in the dataset.
- `client_tenure_years (int)`: The number of years a client has been associated with the company. For example, a client with client_tenure_years = 6 has been with the company for 6 years.
- `client_tenure_months (int)`: The number of months a client has been associated with the company. This value is often more granular than client_tenure_years and could be used for more detailed analysis. For instance, a tenure of 6 years and 1 month would be represented as 73 months.
- `client_age (int)`: The age of the client in years.
- `gender (object)`: The gender of the client. The value can be "Male," "Female," or "Unspecified," meaning the gender data is either recorded or missing.
- `num_accounts (int)`: The number of accounts the client has with the company.
- `balance (float)`: The total balance of the client's accounts with the company. This is a monetary value, and the balance can indicate how much money the client holds across their accounts.
- `calls_last_6_months (int)`: The number of calls the client has made to the company in the past six months. This can give an idea of how actively the client has engaged with the company.
- `logons_last_6_months (int)`: The number of times the client has logged into their account or interacted with the company online in the past six months.

**Observations:**
- This DF has no empty values.
- 14 rows were dropped during the cleaning process, going from 70609 to 70594 rows.